In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# Load ALL samples
df = pd.read_csv('C:/Users/asus/Downloads/archive/fer2013.csv')
X = np.array([np.fromstring(p, sep=' ') for p in df['pixels']]).reshape(-1, 48*48) / 255.0
y = to_categorical(df['emotion'])
print("Loaded all samples - X shape:", X.shape) 


Loaded all samples - X shape: (35887, 2304)


In [2]:
model = Sequential([
    Dense(7, activation='softmax', input_shape=(48*48,))
])
model.compile(optimizer='adam', loss='categorical_crossentropy')
print("Model ready!")

Model ready!


In [3]:
model.fit(X, y, epochs=50, batch_size=12)
print("Training complete!")

Epoch 1/50
2991/2991 [==============================] - 6s 2ms/step - loss: 1.8525
Epoch 2/50
2991/2991 [==============================] - 4s 1ms/step - loss: 1.7992
Epoch 3/50
2991/2991 [==============================] - 4s 1ms/step - loss: 1.7748
Epoch 4/50
2991/2991 [==============================] - 4s 1ms/step - loss: 1.7771
Epoch 5/50
2991/2991 [==============================] - 3s 1ms/step - loss: 1.7509
Epoch 6/50
2991/2991 [==============================] - 3s 1ms/step - loss: 1.7578
Epoch 7/50
2991/2991 [==============================] - 3s 1ms/step - loss: 1.7376
Epoch 8/50
2991/2991 [==============================] - 3s 1ms/step - loss: 1.7387
Epoch 9/50
2991/2991 [==============================] - 3s 1ms/step - loss: 1.7265
Epoch 10/50
2991/2991 [==============================] - 3s 877us/step - loss: 1.7300
Epoch 11/50
2991/2991 [==============================] - 3s 877us/step - loss: 1.7333
Epoch 12/50
2991/2991 [==============================] - 3s 880us/step - loss: 1.

In [4]:
model.save('emotion_model.h5')
print("Model saved as 'emotion_model.h5'")

Model saved as 'emotion_model.h5'


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model and fix metrics warning
model = load_model('emotion_model.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy')  # Fixes warning

emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        face = cv2.resize(gray[y:y+h,x:x+w], (48,48))
        face = face.reshape(1, 48*48) / 255.0  # FLATTEN to (1,2304) 
        
        pred = model.predict(face, verbose=0)[0]
        emotion = emotions[np.argmax(pred)]
        
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(frame, f"{emotion}", (x,y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    
    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()